<a href="https://colab.research.google.com/github/marmb47/MachineLearning/blob/master/regression_lineaire_avec_plusieurs_variables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

Cet article est le troisième de la série intitulée "Machine Learning pour débutant". Dans ce tutoriel, nous allons parler de la régression linéaire avec plusieures variables plus connu sous le nom de "Multivariate linear regression" . Pour ce faire, aprés avoir explorer les données, nous allons représenter le modèle et la fonction d'erreur avant d'entrainer le modèle et de trouver l'équation normale.

# 1- Exploration des données

Dans cet article, Nous allons réaliser un modèle qui permet de prédire le prix d’une maison à partir de sa surface en metre carré et du nombre de chambres. Pour ce faire, nous disposons d’un dataset qui contient les informations (surface, nombre de chambres, et prix) de plusieurs maison.

Le dataset a été téléchargé depuis Coursera. Il s’agit d’un fichier csv(houses-pricing.txt) qui contient trois champs qui portent sur les  différentes maisons. Chaque ligne représente une maison.

Les trois champs sont :

*   Le premier champ représente la surface de la maison en metre carré.
*   Le deuxième champ représente le nombre de chambres de la maison.
*   Le troisième champ représente le prix de la maison en (1000$)

Maintenant, recupérons le dataset pour voir à quoi ils ressemblent.

In [5]:
from google.colab import files
files.upload()

import numpy as np
tab = np.genfromtxt('houses-pricing.txt', delimiter=',')

# recuperation des surfaces des maisons
x1 = tab[:,0];
 
# recuperation des nombres de chambres pour chaque maisons
x2 = tab[:,1];

# recuperation des prix des maisons
Y = tab[:,2];

 
dataset = np.ones((np.size(x1),3))
dataset[:,0] = x1
dataset[:,1] = x2
dataset[:,2] = Y
 
# on affiche les 10 premiers maisons
print(dataset[0:9,:])

Saving houses-pricing.txt to houses-pricing (4).txt
[[2.10400e+03 3.00000e+00 3.99900e+05]
 [1.60000e+03 3.00000e+00 3.29900e+05]
 [2.40000e+03 3.00000e+00 3.69000e+05]
 [1.41600e+03 2.00000e+00 2.32000e+05]
 [3.00000e+03 4.00000e+00 5.39900e+05]
 [1.98500e+03 4.00000e+00 2.99900e+05]
 [1.53400e+03 3.00000e+00 3.14900e+05]
 [1.42700e+03 3.00000e+00 1.98999e+05]
 [1.38000e+03 3.00000e+00 2.12000e+05]]


On a recupéré les surfaces des maisons (x1), les nombres de chambres pour chaque maisons (x2) et les prix des maisons correspondantes (Y) qu’on a mis sur un tableau à trois colonnes qui se nomme “dataset”.

On nomme m le nombre d’exemples (de maisons) dans le dataset. On obtient ainsi l’ensemble $$E_m = \{(x^{(1)},y^{(1)}),(x^{(2)},y^{(2)}),...,(x^{(m)},y^{(m)})\} $$

$$avec \: \: \: \: x^{(i)}=\begin{pmatrix}
x^{(i)}_{1}\\ 
x^{(i)}_{2}
\end{pmatrix} \: \forall \: 0 < i \leq m $$

De ce fait $$ (x^{(i)},y^{(i)}) $$ représente la i-ème ligne de la table “dataset” :  
* la surface $$ x^{(i)}_1 $$ de la i-ème maison
* le nombre de chambres $$ x^{(i)}_2 $$ de la i-ème maison 
* et le prix $$ y^{(i)} $$ de la i-ème maison.

Ici, X est un vecteur de dimension 2 contrairement à l'article précédent ou X était un sacalaire. ce qui veut dire qu'on fait une régréssion avec 2 variables.

Si on représente vectoriellement tout le dataset, on aura :

$$ \forall \: x^{(i)} \in \: X, \: on \: aura \: \: x^{(i)}=\begin{pmatrix}
x^{(i)}_{0}\\ 
x^{(i)}_{1}\\ 
x^{(i)}_{2}
\end{pmatrix} \: avec \: x^{(i)}_{0} = 1 $$

L’ensemble X va devenir alors : $$ X = \begin{pmatrix}
1 & x^{(1)}_{1} & x^{(1)}_{2}\\ 
1 & x^{(2)}_{1} & x^{(2)}_{2}\\ 
. & . & .\\
. & . & .\\
1 & x^{(m)}_{1} & x^{(m)}_{2}
\end{pmatrix} $$

on modifie la varable dataset afin d’ajouter une colonne (avec des 1) devant. ce qui va nous donner :

In [6]:
datasetTemp = dataset
 
dataset = np.ones((np.size(x1),4))
dataset[:,1] = datasetTemp[:,0]
dataset[:,2] = datasetTemp[:,1]
dataset[:,3] = datasetTemp[:,2]
 
X = dataset[:,0:3]
Y = dataset[:,3]
 
print(X.shape)
print(X)
print(Y.shape)
print(Y)

(47, 3)
[[1.000e+00 2.104e+03 3.000e+00]
 [1.000e+00 1.600e+03 3.000e+00]
 [1.000e+00 2.400e+03 3.000e+00]
 [1.000e+00 1.416e+03 2.000e+00]
 [1.000e+00 3.000e+03 4.000e+00]
 [1.000e+00 1.985e+03 4.000e+00]
 [1.000e+00 1.534e+03 3.000e+00]
 [1.000e+00 1.427e+03 3.000e+00]
 [1.000e+00 1.380e+03 3.000e+00]
 [1.000e+00 1.494e+03 3.000e+00]
 [1.000e+00 1.940e+03 4.000e+00]
 [1.000e+00 2.000e+03 3.000e+00]
 [1.000e+00 1.890e+03 3.000e+00]
 [1.000e+00 4.478e+03 5.000e+00]
 [1.000e+00 1.268e+03 3.000e+00]
 [1.000e+00 2.300e+03 4.000e+00]
 [1.000e+00 1.320e+03 2.000e+00]
 [1.000e+00 1.236e+03 3.000e+00]
 [1.000e+00 2.609e+03 4.000e+00]
 [1.000e+00 3.031e+03 4.000e+00]
 [1.000e+00 1.767e+03 3.000e+00]
 [1.000e+00 1.888e+03 2.000e+00]
 [1.000e+00 1.604e+03 3.000e+00]
 [1.000e+00 1.962e+03 4.000e+00]
 [1.000e+00 3.890e+03 3.000e+00]
 [1.000e+00 1.100e+03 3.000e+00]
 [1.000e+00 1.458e+03 3.000e+00]
 [1.000e+00 2.526e+03 3.000e+00]
 [1.000e+00 2.200e+03 3.000e+00]
 [1.000e+00 2.637e+03 3.000e+00]
 [

# 2 - Représentation du modèle et fonction d'erreur

# 3 - Entrainement du modèle

## 3.1 - Descente de gradient

## 3.2 - Normalisation

# 4 - Regréssion polynomiale

# 5 - Equation normale